 # А8.1.4 Наши гипотезы
 
Формулируем наши задачи и гипотезы

Давайте разберёмся, чего именно хочет от нас Сергей.

 1. Определить самые распространенные пути прохождения этапов в приложении – это позволит команде понять, как пользователи пользуются приложением, соотнести идеальный путь пользователей (описанный выше) с фактическим.
 2. Посмотреть на среднее время между различными этапами: выделив этапы, которые занимают больше всего времени, мы сможем их оптимизировать.
 3. Проанализировать зависимость оплат от прохождения обучения. Для этого нам нужно будет определить существует ли различие в частоте и средней величине оплат между 3 группами пользователей:
 - Пользователями, которые прошли обучение хотя бы раз.
 - Пользователями, которые начали обучение, но не прошли его ни разу.
 - Пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

Все примеры будут приводиться на основе данных пользователей, зарегистрировавшихся в 2016 году (с 1 января по 31 декабря 2016 года включительно).

Чтобы вы могли попрактиковать каждый шаг анализа, мы просим вас решать задания на другой выборке пользователей — зарегистрировавшихся в 2017 году. Период анализа — с 1 января по 31 декабря 2017 года.

 # А8.2.1 Импорт библиотек
 
Импорт библиотек

Сначала импортируем нужные библиотеки.

In [2]:
!pip install psycopg2  

In [3]:
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np

Первым делом нам нужно получить данные из базы данных. Сделать мы это можем с помощью библиотеки psycopg2, которая позволяет делать запросы к БД PostgreSQL из Python.

Напишем две функции:

Функция getEventsData() получает данные по событиям пользователей, которые зарегистрировались в 2017 году.

Функция getPurchaseData() получает данные по оплатам пользователей, которые зарегистрировались в 2017 году.

Обратите внимание на параметр курсора cursor_factory=psycopg2.extras.DictCursor. Благодаря этому параметру мы можем получать данные из базы в формате списка словарей. Потом очень удобно будет загрузить такие данные в датафрейм. Результат, возвращаемый каждой функцией после исполнения, запишем в датафреймы events_df и purchase_df.

In [7]:
def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())

In [8]:
events_df.head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,id
0,registration,None,2017-01-01 03:48:40,NaN,27832,80308
1,registration,None,2017-01-01 04:07:25,NaN,27833,80309
2,registration,None,2017-01-01 08:35:10,NaN,27834,80310
3,registration,None,2017-01-01 11:54:47,NaN,27835,80311
4,registration,None,2017-01-01 13:28:07,NaN,27836,80312
5,registration,None,2017-01-01 14:08:40,NaN,27837,80313
6,registration,None,2017-01-01 14:42:58,NaN,27838,80314
7,tutorial_start,None,2017-01-01 14:54:40,31505.0,27836,80315
8,tutorial_start,None,2017-01-01 15:00:51,31506.0,27835,80316
9,tutorial_finish,None,2017-01-01 15:06:15,31506.0,27835,80317


Мы, конечно, можем уже что-то сказать по поводу данных, посмотрев на первые 10 строчек, но лучше использовать более точные методы анализа. Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме, для того, чтобы оценить какие данные содержаться в датафрейме events_df:

In [9]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66959 entries, 0 to 66958
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      66959 non-null  object        
 1   selected_level  8342 non-null   object        
 2   start_time      66959 non-null  datetime64[ns]
 3   tutorial_id     32954 non-null  float64       
 4   user_id         66959 non-null  int64         
 5   id              66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.1+ MB


Итак, мы видим, что датафрейм events_df содержит 66959 строк и 6 столбцов. При этом все строки содержат значения в столбцах id, user_id, start_time, event_type, а остальные столбцы содержат пропущенные значения, например столбец selected_level содержит непустые значения только в 8342 строках.

Исследуем пропуски

Наличие пропущенных строк обусловлено тем, что не все из параметров selected_level, tutorial_id присутствуют в каждом из событий. Эти параметры заполнятся в зависимости от event_type. К примеру, посмотрим на events_df, если оставить в нем только такие строки, где event_type = level_choice:

In [10]:
events_df[events_df['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 19 to 66957
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      8342 non-null   object        
 1   selected_level  8342 non-null   object        
 2   start_time      8342 non-null   datetime64[ns]
 3   tutorial_id     0 non-null      float64       
 4   user_id         8342 non-null   int64         
 5   id              8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB


Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level, но зато содержит пропуски в tutorial_id. Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id.

Теперь проверим аналогичные данные, но при условии, что срез будет содержать данные о событиях tutorial_start и tutorial_finish:

In [11]:
events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 7 to 66955
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      32954 non-null  object        
 1   selected_level  0 non-null      object        
 2   start_time      32954 non-null  datetime64[ns]
 3   tutorial_id     32954 non-null  float64       
 4   user_id         32954 non-null  int64         
 5   id              32954 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.8+ MB


Столбец selected_level в таком срезе не содержит ни одного значения. Это связано с тем, что для событий, связанных с обучением, отсутствуют параметры выбора уровня сложности selected_level. А вот остальные столбцы заполнены полностью.

Исследуем значения

Вызовем метод describe() для того, чтобы оценить характеристики каждого столбца. По умолчанию метод describe() выдает характеристики только по столбцам с численными типами (например, int64, float64). Мы вызовем этот метод с параметром include='all' для того, чтобы отображать характеристики для всех столбцов.

In [12]:
events_df.describe(include='all')

,event_type,selected_level,start_time,tutorial_id,user_id,id
count,66959,8342,66959,32954.000000,66959.000000,66959.000000
unique,5,3,66809,NaN,NaN,NaN
top,registration,medium,2017-03-13 09:37:43,NaN,NaN,NaN
freq,19926,4645,9,NaN,NaN,NaN
first,NaN,NaN,2017-01-01 03:48:40,NaN,NaN,NaN
last,NaN,NaN,2018-01-01 05:50:36,NaN,NaN,NaN
mean,NaN,NaN,NaN,40532.934393,37781.543362,113787.000045
std,NaN,NaN,NaN,5213.486632,5751.497904,19329.542752
min,NaN,NaN,NaN,31505.000000,27832.000000,80308.000000
25%,NaN,NaN,NaN,36008.250000,32849.000000,97047.500000


Давайте оценим, какие уникальные события есть в колонках event_type и selected_level:

In [13]:
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'training_choice'], dtype=object)

In [14]:
events_df['selected_level'].unique()


array([None, 'medium', 'hard', 'easy'], dtype=object)

Также оценим, какое количество пользователей совершали события:

In [15]:
events_df['user_id'].nunique()

19926

 # А8.2.3 Обзор датафрейма с оплатами

 Обзор датафрейма с оплатами

Теперь нам нужно проверить, как события, совершенные пользователями, связаны с последующими оплатами тренировок. Выведем первые 10 строк датафрейма purchase_df с помощью команды head():

In [16]:
purchase_df.head(10)

,user_id,start_time,amount,id
0,37878,2017-06-30 17:05:21,150,17668
1,47216,2017-12-22 06:30:31,25,18396
2,35532,2017-05-21 04:23:32,150,17475
3,42583,2017-10-17 13:52:14,100,18027
4,32321,2017-03-20 06:51:27,150,17202
5,34235,2017-04-23 14:50:26,200,17365
6,36986,2017-06-18 11:51:41,50,17610
7,44157,2017-11-15 17:37:02,100,18161
8,45926,2017-12-05 02:15:59,100,18302
9,32628,2017-03-23 15:55:57,50,17225


Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме для того, чтобы оценить, какие данные содержаться в датафрейме purchase_df:



In [18]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     1600 non-null   int64         
 1   start_time  1600 non-null   datetime64[ns]
 2   amount      1600 non-null   int64         
 3   id          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 50.1 KB


Исследуем значения

Снова обратимся к методу describe() для того, чтобы оценить характеристики каждого столбца датафрейма purchase_df:

In [19]:
purchase_df.describe()

,user_id,amount,id
count,1600.000000,1600.000000,1600.000000
mean,37752.766250,110.734375,17645.505625
std,5822.621784,54.696628,462.038637
min,27845.000000,25.000000,16845.000000
25%,32815.750000,50.000000,17245.750000
50%,37633.500000,100.000000,17645.500000
75%,43023.000000,150.000000,18045.250000
max,47742.000000,300.000000,18452.000000


 # А8.3.1 Задача анализа данных
 
Какова наша первоочередная задача?

Перейдем к анализу данных для того, чтобы выполнить первую нашу задачу: исследовать поведение пользователей.

 - Определить самые распространенные пути прохождения этапов в приложении.
 - Определить среднее время между различными этапами.

Это исследование позволит нам понять, как пользователи пользуются приложением, сравнить фактический путь пользователей с тем, который был задуман при разработке приложения.

Когда мы поймем, какие этапы занимают больше всего времени, мы выясним, какие из этапов нужно оптимизировать.

  А8.3.2 Анализ события registration

Событие registration

Для того, чтобы понимать, как пользователи переходят из этапа в этап, на каких этапах возникают сложности, мы должны определить конверсию на каждом из этапов воронки. То есть нам нужно понять, какой процент пользователей переходит с предыдущего этапа на следующий.

Посмотрим на количество пользователей, которые совершают событие registration:

In [22]:
events_df[events_df['event_type']=='registration']['user_id'].nunique()

19926

In [23]:
events_df['user_id'].nunique()

19926


Как мы видим, все пользователи совершают событие registration. Это обусловлено тем, что этот этап является обязательным для выполнения и без него пользователь не сможет перейти дальше.

А8.3.3 Анализ событий tutorial_start и tutorial_finish

Событие tutorial_start

Посмотрим на срез данных по событию tutorial_start:

In [25]:
events_df[events_df['event_type']=='tutorial_start'].head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,id
7,tutorial_start,None,2017-01-01 14:54:40,31505.0,27836,80315
8,tutorial_start,None,2017-01-01 15:00:51,31506.0,27835,80316
10,tutorial_start,None,2017-01-01 15:40:43,31507.0,27836,80318
12,tutorial_start,None,2017-01-01 17:47:40,31508.0,27833,80320
15,tutorial_start,None,2017-01-01 19:11:36,31509.0,27839,80323
17,tutorial_start,None,2017-01-01 19:46:11,31510.0,27834,80325
29,tutorial_start,None,2017-01-02 02:07:07,31511.0,27840,80337
30,tutorial_start,None,2017-01-02 03:03:44,31512.0,27845,80338
32,tutorial_start,None,2017-01-02 04:55:11,31513.0,27842,80340
41,tutorial_start,None,2017-01-02 07:08:00,31514.0,27845,80349


Посмотрим на количество пользователей, которые совершают событие tutorial_start:

In [26]:
events_df[events_df['event_type']=='tutorial_start']['user_id'].nunique()

11858

Исследуем пользователей, перешедших к обучению

Как мы видим, число пользователей, которые перешли к выполнению обучения, меньше, чем число пользователей, прошедших регистрацию. Давайте определим процент пользователей, которые перешли к выполнению обучения и запишем его в переменную percent_tutorial_start_users:

In [27]:
registered_users_count = events_df[events_df['event_type'] == 'registration']['user_id'].nunique()
tutorial_start_users_count = events_df[events_df['event_type'] == 'tutorial_start']['user_id'].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print ('Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}'.format(percent_tutorial_start_users))

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


Событие tutorial_finish

Теперь давайте посмотрим какое количество пользователей проходит обучение до конца (событие tutorial_finish):

In [28]:
events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].nunique()

10250

Рассчитаем процент пользователей, завершивших обучение, среди пользователей, которые начали обучение. Это будет показатель tutorial_completion_rate (коэффициент завершаемости обучения):

In [29]:
tutorial_finish_users_count = events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print ('Процент пользователей, завершивших обучение: {:.2%}'.format(tutorial_completion_rate))


Процент пользователей, завершивших обучение: 86.44%


Что мы узнали на этом этапе?

В нашем приложении достаточно хороший процент прохождения обучения. Но есть куда стремиться для его увеличения. Подумайте о том, как показатель прохождения может влиять на весь путь пользователя в дальнейшем.

 А8.3.4 Анализ события level_choice
 
Событие level_choice

Следующий этап — это выбор уровня сложности level_choice. Давайте посмотрим на процент тех, кто доходит до этого этапа.

In [30]:
events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()

8342

Из логики нашего приложения (описанной в начале), следует, что событие выбора уровня сложности тренировок можно совершить и без прохождения обучения. Поэтому рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень сложности тренировок, от общего числа зарегистрировавшихся:

In [31]:
level_choice_users_count = events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print ('Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}'.format(percent_level_choice_users))

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


Таким образом, меньше половины пользователей (41,76 %) доходят до этапа выбора уровня сложности тренировок. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные тренировки, которые в дальнейшем могут привести к оплате. Так что оптимизировать прохождение до этого этапа крайне важно для успешной монетизации приложения.